In [192]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim

from sklearn.preprocessing import StandardScaler

import numpy as np

In [193]:
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [194]:
# define input sequence
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110 ,120, 130, 140 ,150, 160, 170,180 ,190, 200]
# choose a number of time steps
n_steps = 3
n_features = 1
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# summarize the data
for i in range(len(X)):
    print(X[i], y[i])

[10 20 30] 40
[20 30 40] 50
[30 40 50] 60
[40 50 60] 70
[50 60 70] 80
[60 70 80] 90
[70 80 90] 100
[ 80  90 100] 110
[ 90 100 110] 120
[100 110 120] 130
[110 120 130] 140
[120 130 140] 150
[130 140 150] 160
[140 150 160] 170
[150 160 170] 180
[160 170 180] 190
[170 180 190] 200


In [204]:
class Dataset(torch.utils.data.Dataset):
    '''
    Prepare the Boston dataset for regression
    '''
    def __init__(self, X, y, scale_data=True):
        self.X = torch.from_numpy(StandardScaler().fit_transform(X))
        media = np.mean(y)
        dt = y.std(ddof=0)
        self.y = torch.from_numpy((y-media)/dt)

    def __len__(self):
          return len(self.X)

    def __getitem__(self, i):
        return self.X[i], self.y[i]

In [205]:
print(X.shape)

(17, 3)


In [206]:
dataset = Dataset(np.array(X),np.array(y))
dataset

In [207]:
trainloader = torch.utils.data.DataLoader(dataset, batch_size=5, shuffle=True, num_workers=1)

In [208]:
class Net(nn.Module):

    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.BatchNorm1d(3),
            nn.Linear(3,64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Linear(64,32),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )

    def forward(self, x):
        x = self.layers(x)
        return x


net = Net().double()
print(net)

Net(
  (layers): Sequential(
    (0): BatchNorm1d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Linear(in_features=3, out_features=64, bias=True)
    (2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=32, bias=True)
    (5): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Linear(in_features=32, out_features=1, bias=True)
  )
)


In [209]:
n_epoch = 5
# Run the training loop

# Funcion de pérdida
criterion = nn.MSELoss()

    
# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)
for epoch in range(0, n_epoch): # 5 epochs at maximum

    # Print epoch
    print(f'Starting epoch {epoch+1}')

    # Set current loss value
    current_loss = 0.0

    # Iterate over the DataLoader for training data
    for i, data in enumerate(trainloader, 0):

        # Get and prepare inputs
        inputs, targets = data
        inputs, targets = inputs.double(), targets.double()
        targets = targets.reshape((targets.shape[0], 1))
        print(targets)
        # Zero the gradients
        optimizer.zero_grad()

        # Perform forward pass
        outputs = net(inputs)
        # Compute loss
        loss = criterion(outputs, targets)
        print(loss)
        # Perform backward pass
        loss.backward()

        # Perform optimization
        optimizer.step()

Starting epoch 1
tensor([[ 0.8165],
        [ 1.0206],
        [-1.0206],
        [-0.2041],
        [-0.6124]], dtype=torch.float64)
tensor(1.1138, dtype=torch.float64, grad_fn=<MseLossBackward0>)
tensor([[-1.6330],
        [-1.4289],
        [ 1.4289],
        [ 0.0000],
        [ 0.4082]], dtype=torch.float64)
tensor(1.2329, dtype=torch.float64, grad_fn=<MseLossBackward0>)
tensor([[ 0.2041],
        [ 0.6124],
        [-1.2247],
        [ 1.6330],
        [ 1.2247]], dtype=torch.float64)
tensor(0.3210, dtype=torch.float64, grad_fn=<MseLossBackward0>)
tensor([[-0.8165],
        [-0.4082]], dtype=torch.float64)
tensor(1.0776, dtype=torch.float64, grad_fn=<MseLossBackward0>)
Starting epoch 2
tensor([[-1.6330],
        [ 1.0206],
        [-0.4082],
        [ 1.6330],
        [-1.4289]], dtype=torch.float64)
tensor(0.6617, dtype=torch.float64, grad_fn=<MseLossBackward0>)
tensor([[-1.2247],
        [ 0.4082],
        [-0.6124],
        [ 0.8165],
        [ 1.4289]], dtype=torch.float64)
t

In [214]:
test = [[180 ,190, 200],[180 ,190, 200],[180 ,190, 200]]
prediccion = [110]

tensor_test = torch.from_numpy(np.array(test)).double()
print(tensor_test)
tensor_prediccion = torch.from_numpy(np.array(prediccion)).double()
tensor_prediccion

tensor([[ 80.,  90., 100.]], dtype=torch.float64)


tensor([110.], dtype=torch.float64)

In [215]:
net(tensor_test)

ValueError: Expected more than 1 value per channel when training, got input size torch.Size([1, 3])